In [7]:
# Import functions 
import sys
sys.path.insert(0, '..')
from utils import *

# Time limit for debugging video
time_limit = 10    

In [8]:
# Starter code class that handles the fancy stuff. No need to modify this! 
class Racecar:
    SCAN_TOPIC = "/scan"
    IMAGE_TOPIC = "/camera"
    DRIVE_TOPIC = "/drive"
    
    def __init__(self):
        self.sub_scan = rospy.Subscriber(self.SCAN_TOPIC, LaserScan, callback=self.scan_callback)
        self.sub_image = rospy.Subscriber(self.IMAGE_TOPIC, Image, callback=self.image_callback)
        self.pub_drive = rospy.Publisher(self.DRIVE_TOPIC, AckermannDriveStamped, queue_size=1)
        self.last_drive = AckermannDriveStamped()
    
    def image_callback(self, msg):
        self.last_image = msg.data
        
    def show_last_image(self):
        im = np.fromstring(self.last_image,dtype=np.uint8).reshape((480,-1,3))[...,::-1]
        return im
        
    def scan_callback(self, msg):
        self.last_scan = msg.ranges
        
    def drive(self, speed, angle):
        msg = AckermannDriveStamped()
        msg.drive.speed = speed
        msg.drive.steering_angle = angle
        self.last_drive = msg
    
    def stop(self):
        self.drive(0, 0) #self.last_drive.drive.steering_angle)
    
    def look(self):
        return self.last_image
    
    def scan(self):
        return self.last_scan
    
    def run(self, func, limit = 10):
        r = rospy.Rate(60)
        t = rospy.get_time()
        video = cv2.VideoCapture(1)
        while rospy.get_time() - t < limit and not rospy.is_shutdown():
            func(video.read()[1])
            self.pub_drive.publish(self.last_drive)
            r.sleep()
        video.release()
        
        self.stop()
        self.pub_drive.publish(self.last_drive)
        time.sleep(0.1)
    
rospy.init_node('racecar')
rc = Racecar()
print('ROS node started successfully')

ROS node started successfully


<span style='font-size:1.75rem;line-height:1.5'>
   <center>
      <h3> We are getting our cars today!</h3>
   </center>
    <h4 style='color:red;'> Before we start setting up, make sure to review the rules: </h4>
    <ol>
        <li> Make sure that a car is always on a BLOCK before testing code the first time. </li>
        <li> To run on the ground, your car NEEDS A STICKER from an instructor/TA.  </li>
        <img src='img1.jpg' height='800' width='800'> </img> <br/>
    </ol>
    <center style='color:red;'> EACH TIME THE CAR CRASHES (PHYSICALLY), WE WILL TAKE YOUR CAR AWAY <br/> 
    FOR A CERTAIN AMOUNT (MAY BE 10-20 MINUTES OR EVEN A WHOLE DAY) AND DECREASE THE SPEED. </center>
    <center style='color:green;'> Right now, our cars are at 85% possible speed. If our class is good and not crashing, we may increase the cars to the max speed for the final race. </center>
</span>

<span style='font-size:1.75rem;line-height:1.5'>
    To have our cars running, we need to do a few things first: 
   <ol>
      <li> We need to make sure that our car brain is turned on by plugging in the white battery that powers the brains:
         <img src='whitebat1.jpg' height='300' width='300' />
         This is how it looks when it plugs in:
         <img src='whitebat2.jpg' height='500' width='300' style='transform:rotate(90deg);' /> <br/> <br/>
         When this battery is plugged in, the jetson (brain) should light up here, like this:
         <img src='whitebat3.jpg' height='300' width='300' />
      </li>
      <li> We also need to turn our car motor on by plugging this battery in:  
         <img src='motbat1.jpg' height='300' width='300' style='transform:rotate(180deg);'/> 
         This battery needs to be connected to here in the car:
         <img src='motbat2.jpg' height='300' width='300' /> 
         <img src='motbat3.jpg' height='300' width='300' />
         This is how it looks when it's connected and secured in place:
         <img src='motbat4.jpg' height='300' width='300' />
         We also need to make sure that this switch is turned on: <br/> <br/>
         <img src='switch.jpg' height='300' width='300' style='transform:rotate(90deg);'/> <br/> <br/>
      </li>
      Now that the hardware is all ready to go, let us access the software part of the car. 
      <li>
         We can access the software through the car's jupyter notebook remotely from the computer. 
         To do so, first make sure that the computer is on the correct wifi for the car we are trying to connect:
         <ul>
            <li> RMS_BRAVO: 2, 3, 4, 5 </li>
            <li> RMS_CHARLIE: 6, 7, 8, 9  </li>
            <li> RMS_DELTA: 10, 11, 12, 13 </li>
         </ul>
      </li>
      <li> Once we have our computers on the right wifi, open the google chrome web browser on your computer. It should automatically redirect you to the correct jupyter notebook. If not, type <code>192.168.1.&lt;car#&gt;:8888 </code> 
          <img src='' height='400' width='800'/>
      </li>
      <li> From there, jupyter should open. <br>
          Then, we will be able to get to the car's jupyter notebook. 
          Now, we need to open the terminal from the jupyter home screen. 
          <img src="open_term.png" height='400' width='800'/>
          <img src="term1.png" height='400' width='800'/> <br> 
      </li>
      <li>  Then, type <code> bash </code> into the terminal. 
            This should happen: <img src="term2.png" height='400' width='800'/> <br> 
            Then, type in <code> teleop. </code> 
            This should happen: <img src="term3.png" height='400' width='800'/> 
            When teleop is starting to run, make sure to have the joystick (see below for an image) nearby and press its center button. 
            <br> <br> <br>  <img src='joystick.jpg' height='300' width='300' style='transform:rotate(90deg);'/> <br> <br> 
      </li>
      <li> Now, we can control the racecar with our controllers and also run almost any code here. Test the following code below to make sure the car runs. <br> <p style='color:red'> MAKE SURE THE CAR IS ON A BLOCK BEFORE RUNNING. </p> <img src='img1.jpg' height='300' width='300'/> The car should start driving straight forward. </li>
   </ol>
</span>

In [ ]:
# car will drive forward for ~10 seconds
def drive_forward(frame):
    rc.drive(0.2, 0)
    
rc.run(drive_forward)

<span style='font-size:1.75rem;line-height:1.5'>
    <code>rc.drive({speed}, {angle})</code> is a function that tells the car to drive at the speed and angle passed into the function. 
    <code> rc.run({function name})</code> is a function that tells the car to run the function passed into <code>rc.run</code> for about 10 seconds. 
     For now, let us experiment with the speed and angle to get used to how the car measurements work. 
</span>

In [5]:
# car will drive forward for ~10 seconds
def drive_forward(frame):
    rc.drive(0.2, 0) # change these numbers 
    
rc.run(drive_forward)

END OF ROSPY RUN


# Part 1: 

<span style='font-size:1.75rem;line-height:1.5'>
By now, we have realized that we need approximate what '0.2' in car measurements translates into human measurements. To do so, we TAs have prepared a long ruler tape that the car can drive upon and based on that and a (possibly human) calculator, we can calculate the speed of car in in/s. 
    
<img src='' height='300' width='300'/>

Car measurements and human measurements may not translate linearly so it may be wise to test at different intervals. 

To help both us document and double check (as well as your understanding of exactly how fast the cars can run), submit the results to this form: https://forms.gle/iNzNW2aiQE8nFAqA7 (You may submit it may times) 

If the measurements are drastically different from what is expected, we will tell you and fix it (may be a low motor battery problem). 

Get a TA's okay when your group is ready to start the car on the ground. 
(TAs may ask you at what number a car will no longer speed up or turn any more.)

<p style='color:red'> 
    You MUST get a TA's approval before taking your car off the block and driving your car on the floor. If we see cars driving without the appropriate sticker, your racecar will be taken away.
    </p>
</span>

In [ ]:
def drive_forward(frame):
    rc.run(0.2, 0)
    
rc.run(drive_forward)

# Part 2
---- 

<span style='font-size:1.75rem;line-height:1.5'>

Now that we have basic idea of the measurements, let us make a function that will drive the car in a circle autonomously! 
</span>

In [9]:
# define your own function 

def circle(frame):
    pass 
    # use the drive function to have the car run in a circle 
    rc.drive(0.2, 0)

rc.run(circle)

<span style='font-size:1.75rem;line-height:1.5'>
Now, for our next challenge, we will be learning a new racecar function:

<code>rc.run({function name}, {arguments})</code> is a function that tells the car to run the {function} passed into <code>rc.run</code> for about 10 seconds and also passes {arguments} to {function}. Basically think of it as running function(frame, {arguments}) for 10 seconds. (Note: it is actually arguments being passed, not {arguments}). 

Now, using the function we have just learned, try using making the car drive in a square autonomously! 
</span> 

In [ ]:
def square(frame, arguments):
    pass 
    # use the drive function to have the car run 

# rc.run()

<span style='font-size:1.75rem;line-height:1.5'>

Now try driving it in a triangle autonomously! 
</span> 

In [ ]:
def triangle(frame, arguments):
    pass 
    # use the drive function to have the car run 

# rc.run()

<span style='font-size:1.75rem;line-height:1.5'>

Challenge: try driving it in a figure-eight autonomously! 
</span> 

In [ ]:
def figure_eight(frame, arguments):
    pass 
    # use the drive function to have the car run 

# rc.run()

# Mini Competition 1: Cone Parking

<span style='font-size:1.75rem;line-height:1.5'>

Now, we're going to have a small competition where we learn to park cars in front of a cone. Here are the rules and how points are counted:
<ol>
    <li> Winner is car that starts the furthest away from cone, and parks exactly half a foot away from the cone. </li> 
    <li> -10 points will be subtracted from each attempt that the car bumps/touches into the cone. </li>
    <li> -20 points will be subtracted from each attempt that the car crashes into the cone. </li>
    <li> Hhitting a wall = car getting taken away, so BE CAREFUL EVEN ON YOUR FIRST ATTEMPTS. It is better to start off too slow than too fast. <b>START SLOW</b></li>
    <li> Positive points calculation formula (best attempt): starting distance from cone*speed of car - abs(ending distance from a cone - .5)  </li> 
    <li> Any attempt that involves the joystick taking over control will be disqualified </li>
    <li> Final point value = best attempt positive value - abs(negative point value) </li>
</ol>
</span>

In [ ]:
def known_cone_parking(frame, argument):
    pass 

# rc.run() 


Challenge: Based on a distance given in feet, create an algorithm (code) that will park a feet from the distance. 

# Competition 2: Obstacle Navigation

<span style='font-size:1.75rem;line-height:1.5'>

Next, we will have a competition to see who can best navigate obstacles of exactly 4 feet apart. The winning car will be the car that can best weave through these obstacles and not bump into them. This time, we won't give a rubric.

Have fun testing! 
<img src='https://sayingimages.com/wp-content/uploads/dont-worry-you-got-this-meme.jpg' height='300' width='300'/> 

</span>


In [ ]:
def known_obs_nav(frame, arguments):
    pass 

# rc.run() 
